In [0]:
from datetime import datetime, date, time
from os.path import *
import pandas as pd
import os

In [0]:
dataAtual = date.today()
print(dataAtual.strftime('%Y%m%d'))

20220318


In [0]:
lista_tabelas = ['product_vendor', 'purchase_order_detail', 'purchase_order_header', 'ship_method', 'vendor']

In [0]:
def lerArquivos(tables):
    container= 'bronze'
    origem= 'purchasing'
    dicionarioDfs = {}
    for table in tables:
        dicionarioDfs[table] = spark.read.parquet("/mnt/" + container + "/" + origem + "/" + table + "/" + str(dataAtual.strftime('%Y%m%d')))
    return dicionarioDfs

In [0]:
dfs = lerArquivos(lista_tabelas)


In [0]:
def registrar(dicionario):
    for key, value in dicionario.items(): #Key = chave, o nome da chave será a chave do dicionario | Value = informacoes de cada chave | Items = listagem key e values
        value.registerTempTable(key)
        print('Tabela registrada: ' + key)

In [0]:
registrar(dfs)

Tabela registrada: product_vendor
Tabela registrada: purchase_order_detail
Tabela registrada: purchase_order_header
Tabela registrada: ship_method
Tabela registrada: vendor


In [0]:
def gravarArquivos(dicionario, arquivo):
    container= 'silver'
    origem= 'purchasing'
    pasta = 'Logs'
    formatoDataHora = '%Y%m%d%H%M%S'
    dataHoje = str(datetime.today().strftime(formatoDataHora))
    msg = ''
    for key, value in dicionario.items():
        try:
            value.write.format("Delta").save("/mnt/" + container + "/" + origem + "/" + key)
            mensagem1 = 'Tabela registrada: ' + key
            mensagem2 = 'Quantidade de linhas lidas: ' + str(value.count()) + '.'
        except Exception as e:
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            mensagem1 = 'Tabela não registrada: ' + key
            mensagem2 = 'Zero linhas gravadas.' + template.format(type(e).__name__, e.args)
            
        mensagemFinal = mensagem1 + ". " + mensagem2
        msg = msg + "\n" + mensagemFinal

    dbutils.fs.put("/mnt/" + container + "/" + pasta + "/" + dataHoje + ".txt", msg)
    

In [0]:
gravarArquivos(dfs, "Delta")

Wrote 4210 bytes.
